In [2]:
from collections import defaultdict
import scipy
import scipy.optimize
import numpy as np
import random

# Goal:
Given (user, music, format(optional)) tuple, predict the rating that the user will give to the music.

# Part 0: Util Functions

In [3]:
# useful fields:
# |name       | possible value  | analysis
# "overall":    1 - 5 (int)
# "verified":   True / False      (Don't know meaning yet)
# "reviewerID": "A1SJL3JBBILJ66"
# "asin": "     B0018CGCR4"       (music ID)
# "format":     " MP3 Music"      86.44%
#               " Audio CD"       6.37%
#               "" (undeclared)   6.95%
#               " Vinyl"          .2%
#               (others)          <.04%
# "reviewText": "THANK YOU"       .09% users doesn't provide reviewText, indcicate as ""
# "summary":    "Five Stars"      .002% users doesn't provide summary, indcicate as ""
# "image":      0 (int)           .107% users provide image
#                                 indicate number of images provided in the review
# "vote":       0 (int)           4.48% reviewers are voted by others

In [4]:
%%time

# constrain: ensure each user/data appear at least 4 times in the training set
print('Building training/validation sets... ', end='')

f = open("./train.json", 'rt', encoding="utf8")

train = [eval(l) for l in f]
valid = train[140000:]
train = train[:140000]

f = open("./test.json", 'rt', encoding="utf8")
test = [eval(l) for l in f]

print('Finish\n'
      'Size of train      set: %d\n'
      'Size of validation set: %d\n'
      'Size of test       set: %d\n' % (len(train), len(valid), len(test)))

Building training/validation sets... Finish
Size of train      set: 140000
Size of validation set: 10000
Size of test       set: 19781

CPU times: user 5.78 s, sys: 205 ms, total: 5.99 s
Wall time: 6.08 s


In [5]:
def MAE(pred, actual):
    pass

# Part 1.1 : Naive solution 1, Global Average
Predict Value = $\theta_0$ + $\theta_1$ * (average rating that the user gives out) + $\theta_2$ * (average rating that the music receives)

Use linear regressor to optimize $\theta_0$, $\theta_1$ and $\theta_2$

In [8]:
# TODO
train[0]

{'asin': 'B0018CGCR4',
 'format': ' MP3 Music',
 'image': 0,
 'overall': 5,
 'reviewText': 'THANK YOU',
 'reviewerID': 'A1SJL3JBBILJ66',
 'summary': 'Five Stars',
 'verified': True,
 'vote': 0}

In [50]:
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
users = []
items = []
for d in train:
    r = int(d['overall'])
    ratingsPerUser[d['reviewerID']].append(r)
    ratingsPerItem[d['asin']].append(r)
    users.append(d['reviewerID'])
    items.append(d['asin'])

In [60]:
avgRatingsPerUser = defaultdict(int)
avgRatingsPerItem = defaultdict(int)
for u in ratingsPerUser:
    avgRatingsPerUser[u] = np.mean(ratingsPerUser[u])
for item in ratingsPerItem:
    avgRatingsPerItem[item] = np.mean(ratingsPerItem[item])

trainRatings = [int(d['overall']) for d in train]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)

/Users/alexli/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/alexli/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [61]:
globalAverage

4.700542857142858

In [65]:
def feature(datum):
    feat = [1,avgRatingsPerUser[datum['reviewerID']],avgRatingsPerItem[datum['asin']]] 
    return feat
X_train = [feature(d) for d in train]
y_train = [int(d['overall']) for d in train]

In [66]:
from sklearn import linear_model
mod = linear_model.LogisticRegression(C=1.0)
mod.fit(X_train,y_train)

/Users/alexli/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/alexli/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
X_valid = [feature(d) for d in valid]
y_valid = [int(d['overall']) for d in valid]
valid_predictions = mod.predict(X_valid)
sum(valid_predictions == y_valid) /len(y2_test)

# Part 1.2: Naive Solution 2, tf-idf
Use the similar approach in hw4:

Find tf-idf of reviews, then train a Ridge model to predict the rating. 

In [6]:
# TODO

# Part 2.1: Solution 1

In [7]:
# TODO

# Part 3: Performance

In [8]:
#TODO

We can't prove that user will rate a music he'she never listened as the way he/she will rate a listened music.

For example, a user will only rate musics he/she like, so he/she rated every music 5 stars.

It is possible that our model will predict that the user will give high ratings to all musics.